In [68]:
import spotifyapi
import pandas as pd
import time

To use the Spotify API you must have API tokens

Steps to get API access:
1. Log into the dashboard using your Spotify account @ https://developer.spotify.com/
2. Create an app and select "Web API" for the question asking which APIs are you planning to use. Once you have created your app, you will have access to the app credentials (Client ID and Secret). These will be required for API authorization to obtain an access token.
3. Use the access token in your API requests.

In [7]:
#Note that you should not share your Client ID or Secret with Anyone, but for ease of grading yolo.
Client_ID = "080544c4ba6b4712a6e11a98dfc2028a"
Client_Secret = "93e55c34c76e4dc79b3e722cdddcf276"

In [11]:
#Get OAuth 2.0 Tokens, if your ./data/access_token.json file is editted / corrupted / or formatted incorrectly and you are getting errors, just delete the json file and rerun this function.
access_token = spotifyapi.get_access_token(Client_ID, Client_Secret)

./spotifyapi/data/access_token.json not found. Creating file and getting new token...
API request succeeded!
{'access_token': 'BQBNOWA8dbiaM38iGmigAlFPpQ3Zn_AkfozupKGc0SyXARgOYIfMmOFPb27I9H3KiNVwCkyMrMF8xACSE3w6Ame7q7-vqQ88Sncn2a3aGh5HkZzYSaY', 'token_type': 'Bearer', 'expires_in': 3600}


In [12]:
#Make a list of artists you want to collect songs for
artist_ids = ['3WrFJ7ztbogyGnTHbHJFl2', '3r9TXuXfOUxXjgYgAR0fP8', '0LcJLqbBmaGUft1e9Mm8HV', '6A43Djmhbe9100UwnI7epV', '1HY2Jd0NmPuamShAr6KMms', '3jOstUTkEu2JkjvRdBA5Gu']
market = 'US'

https://api.spotify.com/v1/artists/3WrFJ7ztbogyGnTHbHJFl2/top-tracks?market=US
Getting track data...
API request succeeded!


In [38]:
artist_name = []
artist_id_list = []
track_name = []
track_id_list = []

for artist in artist_ids:
    #sleeping between requests so as to not get timed out by the spotify api. Example should take about 1 minute.
    time.sleep(10)
    response = spotifyapi.get_artist_top_tracks(artist, access_token=access_token, market=market)
    for track in response['tracks']:
        # Extracting the first artist's name
        artist_name_var = track['artists'][0]['name']
        artist_name.append(artist_name_var)

        # Extracting the track name
        track_name_var = track['name']
        track_name.append(track_name_var)

        # Extracting the first artist's id
        artist_id_var = track['artists'][0]['id']
        artist_id_list.append(artist_id_var)

        # Extracting the track id
        track_id_var = track['id']
        track_id_list.append(track_id_var)

track_df = pd.DataFrame({
    'Artist': artist_name,
    'artist_id': artist_id_list,
    'Track Name': track_name,
    'track_id': track_id_list
})

https://api.spotify.com/v1/artists/3WrFJ7ztbogyGnTHbHJFl2/top-tracks?market=US
Getting track data...
API request succeeded!
https://api.spotify.com/v1/artists/3r9TXuXfOUxXjgYgAR0fP8/top-tracks?market=US
Getting track data...
API request succeeded!
https://api.spotify.com/v1/artists/0LcJLqbBmaGUft1e9Mm8HV/top-tracks?market=US
Getting track data...
API request succeeded!
https://api.spotify.com/v1/artists/6A43Djmhbe9100UwnI7epV/top-tracks?market=US
Getting track data...
API request succeeded!
https://api.spotify.com/v1/artists/1HY2Jd0NmPuamShAr6KMms/top-tracks?market=US
Getting track data...
API request succeeded!
https://api.spotify.com/v1/artists/3jOstUTkEu2JkjvRdBA5Gu/top-tracks?market=US
Getting track data...
API request succeeded!


In [51]:
unique_artist_ids = list(set(artist_id_list))
unique_artist_ids

response = spotifyapi.get_artists(unique_artist_ids, access_token)
response

https://api.spotify.com/v1/artists?ids=3WrFJ7ztbogyGnTHbHJFl2,1HY2Jd0NmPuamShAr6KMms,3r9TXuXfOUxXjgYgAR0fP8,6A43Djmhbe9100UwnI7epV,3jOstUTkEu2JkjvRdBA5Gu,0LcJLqbBmaGUft1e9Mm8HV,6USv9qhCn6zfxlBQIYJ9qs
Getting artists data...
API request succeeded!


{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3WrFJ7ztbogyGnTHbHJFl2'},
   'followers': {'href': None, 'total': 26810844},
   'genres': ['british invasion',
    'classic rock',
    'merseybeat',
    'psychedelic rock',
    'rock'],
   'href': 'https://api.spotify.com/v1/artists/3WrFJ7ztbogyGnTHbHJFl2',
   'id': '3WrFJ7ztbogyGnTHbHJFl2',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab6761610000e5ebe9348cc01ff5d55971b22433',
     'width': 640},
    {'height': 320,
     'url': 'https://i.scdn.co/image/ab67616100005174e9348cc01ff5d55971b22433',
     'width': 320},
    {'height': 160,
     'url': 'https://i.scdn.co/image/ab6761610000f178e9348cc01ff5d55971b22433',
     'width': 160}],
   'name': 'The Beatles',
   'popularity': 83,
   'type': 'artist',
   'uri': 'spotify:artist:3WrFJ7ztbogyGnTHbHJFl2'},
  {'external_urls': {'spotify': 'https://open.spotify.com/artist/1HY2Jd0NmPuamShAr6KMms'},
   'followers': {'href': None, 'total': 28822420}

In [62]:
artist_name_column = []
artist_id_column = []
genres_column = []
followers_column = []
popularity_column = []

# Loop through the 'artists' in the response
for artist in response['artists']:
    # Extracting values for each column
    artist_name_column.append(artist['name'])
    artist_id_column.append(artist['id'])
    genres_column.append(','.join(artist['genres']))
    followers_column.append(artist['followers']['total'])
    popularity_column.append(artist['popularity'])

# Creating a DataFrame
artist_df = pd.DataFrame({
    'artist_name': artist_name_column,
    'artist_id': artist_id_column,
    'genres': genres_column,
    'followers': followers_column,
    'popularity': popularity_column
})

In [44]:
audio_features = spotifyapi.get_tracks_audio_features(track_id_list, access_token)

rows = []

for i, track_id in enumerate(track_id_list):
    # Create a dictionary for each row
    row_dict = {"track_ids": track_id}
    
    # Add key-value pairs for each audio feature
    for feature_key, feature_value in audio_features["audio_features"][i].items():
        row_dict[feature_key] = feature_value
    
    # Append the row dictionary to the list
    rows.append(row_dict)

# Create a DataFrame from the list of rows
audio_features_df = pd.DataFrame(rows)

In [70]:
final_df = spotifyapi.join_tables(track_df, artist_df, audio_features_df)
final_df

AttributeError: module 'spotifyapi' has no attribute 'join_tables'